In [1]:
# Import all utility functions
from utility import *
from matplotlib.ticker import LogLocator

## Real Data: Dynamics of at and qt and Delta

In [ ]:
df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data")
df

### Read Data

In [ ]:
df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data")
df = df[['b','30.83','1.25','+']]
df.columns = ['b','x1','x2','y']
# preprocessing
indexDrop = df[df['x1'] == '?'].index
df.drop(indexDrop , inplace=True)
df['x1'] = df['x1'].astype(float)
df = df.replace(['+','-'],[1,-1])
print(df.head())

# normalize
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(df['x1'].to_numpy().reshape(-1, 1))
x1_new = scaler.transform(df['x1'].to_numpy().reshape(-1, 1)).reshape(-1)
scaler.fit(df['x2'].to_numpy().reshape(-1, 1))
x2_new = scaler.transform(df['x2'].to_numpy().reshape(-1, 1)).reshape(-1)
df['x1'] = x1_new
df['x2'] = x2_new
df_a_n = df[(df['y'] == -1) & (df['b'] == 'a')]
df_a_p = df[(df['y'] == 1) & (df['b'] == 'a')] 
df_b_n = df[(df['y'] == -1) & (df['b'] == 'b')]
df_b_p = df[(df['y'] == 1) & (df['b'] == 'b')] 

alpha_a = len(df_a_p)/(len(df_a_p) + len(df_a_n))
alpha_b = len(df_b_p)/(len(df_b_p) + len(df_b_n))
print(alpha_a, alpha_b)

### Fit Beta Distribution

In [4]:
def plot_beta(x_range, a, b, mu=0, sigma=1, cdf=False, **kwargs):
    '''
    Plots the f distribution function for a given x range, a and b
    If mu and sigma are not provided, standard beta is plotted
    If cdf=True cumulative distribution is plotted
    Passes any keyword arguments to matplotlib plot function
    '''
    x = x_range
    if cdf:
        y = beta.cdf(x, a, b, mu, sigma)
    else:
        y = beta.pdf(x, a, b, mu, sigma)
    plt.plot(x, y, **kwargs)

In [ ]:
# save the beta distribution parameters
params_1 = {'a':{'p':[],'n':[]},'b':{'p':[],'n':[]}}
params_2 = {'a':{'p':[],'n':[]},'b':{'p':[],'n':[]}}

# fit parameters
params_1['a'] ['n'] = beta.fit(df_a_n['x1'],floc = -0.001, fscale = 1.01)
params_1['a'] ['p'] = beta.fit(df_a_p['x1'],floc = -0.001, fscale = 1.01)
params_1['b'] ['n'] = beta.fit(df_b_n['x1'],floc = -0.001, fscale = 1.01)
params_1['b'] ['p'] = beta.fit(df_b_p['x1'],floc = -0.001, fscale = 1.01)
params_2['a'] ['n'] = beta.fit(df_a_n['x2'],floc = -0.001, fscale = 1.01)
params_2['a'] ['p'] = beta.fit(df_a_p['x2'],floc = -0.001, fscale = 1.01)
params_2['b'] ['n'] = beta.fit(df_b_n['x2'],floc = -0.001, fscale = 1.01)
params_2['b'] ['p'] = beta.fit(df_b_p['x2'],floc = -0.001, fscale = 1.01)

print(params_1, params_2)

In [ ]:
# plot the distribution
x = np.linspace(0, 1, 500)
fig = plt.figure(figsize=(12, 2.5))

# group a and X1
plt.subplot(1,4,1)
plt.ylim(0.1, 11.6)
plt.xlim(0, 1)
plot_beta(x, params_1['a'] ['n'][0], params_1['a'] ['n'][1], 0, 1, color='blue', lw=2, ls='-')
plot_beta(x, params_1['a'] ['p'][0], params_1['a'] ['p'][1], 0, 1, color='red', lw=2, ls='-')
plt.hist(df_a_n['x1'], density=True, color = 'blue', bins=10, label='unqualified',alpha=0.5)
plt.hist(df_a_p['x1'], density=True, bins=100, color = 'red', label='qualified',alpha=0.5)
plt.title(r"Group $i$")
plt.xlabel(r'$X_1$')
plt.legend()


# group a and X2
plt.subplot(1,4,2)
plt.ylim(0.1, 11.6)
plt.xlim(0, 1)
plot_beta(x, params_2['a'] ['n'][0], params_2['a'] ['n'][1], 0, 1, color='blue', lw=2, ls='-')
plot_beta(x, params_2['a'] ['p'][0], params_2['a'] ['p'][1], 0, 1, color='red', lw=2, ls='-')
plt.hist(df_a_n['x2'], density=True, color = 'blue', bins=10, label='unqualified',alpha=0.5)
plt.hist(df_a_p['x2'], density=True, bins=100, color = 'red', label='qualified',alpha=0.5)
plt.title(r"Group $i$")
plt.xlabel(r'$X_2$')
plt.legend()


# group b and X1
plt.subplot(1,4,3)
plt.ylim(0.1, 11.6)
plt.xlim(0, 1)
plot_beta(x, params_1['b'] ['n'][0], params_1['b'] ['n'][1], 0, 1, color='blue', lw=2, ls='-')
plot_beta(x, params_1['b'] ['p'][0], params_1['b'] ['p'][1], 0, 1, color='red', lw=2, ls='-')
plt.hist(df_b_n['x1'], density=True, color = 'blue', bins=10, label='unqualified',alpha=0.5)
plt.hist(df_b_p['x1'], density=True, bins=100, color = 'red', label='qualified',alpha=0.5)
plt.title(r"Group $j$")
plt.xlabel(r'$X_1$')
plt.legend()


# group b and X2
plt.subplot(1,4,4)
plt.ylim(0.1, 11.6)
plt.xlim(0, 1)
plot_beta(x, params_2['b'] ['n'][0], params_2['b'] ['n'][1], 0, 1, color='blue', lw=2, ls='-')
plot_beta(x, params_2['b'] ['p'][0], params_2['b'] ['p'][1], 0, 1, color='red', lw=2, ls='-')
plt.hist(df_b_n['x2'], density=True, color = 'blue', bins=10, label='unqualified',alpha=0.5)
plt.hist(df_b_p['x2'], density=True, bins=100, color = 'red', label='qualified',alpha=0.5)
plt.title(r"Group $j$")
plt.xlabel(r'$X_2$')
plt.legend()
plt.tight_layout()
# plt.subplots_adjust(left=0.1,
#                     bottom=0.1,
#                     right=0.9,
#                     top=0.9,
#                     wspace=0.2,
#                     hspace=0.4)
plt.show()
fig.savefig('plots_new/feature_dist_real.pdf')

### Verify the monotone likelihood 

In [ ]:
def plot_monotone(x_range, a0, b0, a1, b1, mu=0, sigma=1, **kwargs):
    '''
    Plots the f distribution function for a given x range, a and b
    If mu and sigma are not provided, standard beta is plotted
    If cdf=True cumulative distribution is plotted
    Passes any keyword arguments to matplotlib plot function
    '''
    x = x_range
    y0 = beta.pdf(x, a0, b0, mu, sigma)
    y1 = beta.pdf(x, a1, b1, mu, sigma)
    ratio_10 = y1/y0
    plt.plot(x, ratio_10, **kwargs)
    plt.yscale('log')


fig = plt.figure(figsize=(6,2.5))   
ax = fig.add_subplot(121)
plt.xlim(0, 1)
plt.ylim(0.1, 10**5)
plot_monotone(x,params_1['a'] ['n'][0],params_1['a'] ['n'][1],params_1['a'] ['p'][0],params_1['a'] ['p'][1], 0, 1, lw=1.5, ls='-',label = 'Group a')
plot_monotone(x,params_1['b'] ['n'][0],params_1['b'] ['n'][1],params_1['b'] ['p'][0],params_1['b'] ['p'][1], 0, 1, lw=1.5, ls='-',label = 'Group b')
ax.yaxis.set_major_locator(LogLocator(base=10**5))
ax.set_xlabel(r'$X_1$')
plt.legend()
ax = fig.add_subplot(122)
plt.xlim(0, 1)
plt.ylim(0.1, 10**5)
plot_monotone(x,params_2['a'] ['n'][0],params_2['a'] ['n'][1],params_2['a'] ['p'][0],params_2['a'] ['p'][1], 0, 1, lw=1.5, ls='-',label = 'Group a')
plot_monotone(x,params_2['b'] ['n'][0],params_2['b'] ['n'][1],params_2['b'] ['p'][0],params_2['b'] ['p'][1], 0, 1, lw=1.5, ls='-',label = 'Group b')
ax.yaxis.set_major_locator(LogLocator(base=10**5))
ax.set_xlabel(r'$X_2$')
plt.legend()
plt.minorticks_off()
fig.savefig('plots_new/monotone.pdf', bbox_inches='tight')
plt.show()

### Experiment Begins

- Note: It may take a long time to run 100 trials.
- Note: slight quantitative differences may exist because of deprecated versions of sklearn implement logistic classifier, but the qualitative results should remain same.

In [ ]:
n = 100
T = 15
Q = np.array([[5,0], [0,5]])
N = 2000
alphas = {'a':alpha_a, 'b':alpha_b}
tp=2

Group $i$

In [ ]:
# ratio = 0.1
ratio = 0.1
np.random.seed(42)
r = 0.1
bias = 'up'
mag = 0.1
des = f"real_setting_ratio{r}_bias{bias}_mag{mag}"
At, Qt, At_sd, Qt_sd = simulation(Q,N,n,T,alphas,bias,mag,tp,r,params_1=params_1,params_2=params_2,sd=True)
plot_save_single(At, Qt, des, True)
plot_save_single_err(At, Qt, At_sd, Qt_sd, des, True)

In [ ]:
# ratio = 0.05
np.random.seed(42)
r = 0.05
bias = 'up'
mag = 0.1
des = f"real_setting_ratio{r}_bias{bias}_mag{mag}"
At, Qt, At_sd, Qt_sd = simulation(Q,N,n,T,alphas,bias,mag,tp,r,params_1=params_1,params_2=params_2,sd=True)
plot_save_single(At, Qt, des, True)
plot_save_single_err(At, Qt, At_sd, Qt_sd, des, True)

In [ ]:
# ratio = 0
np.random.seed(42)
r = 0
bias = 'up'
mag = 0.1
des = f"real_setting_ratio{r}_bias{bias}_mag{mag}"
At, Qt, At_sd, Qt_sd = simulation(Q,N,n,T,alphas,bias,mag,tp,r,params_1=params_1,params_2=params_2,sd=True)
plot_save_single(At, Qt, des, True)
plot_save_single_err(At, Qt, At_sd, Qt_sd, des, True)

In [ ]:
#infinite horizon to verify the limit
n = 20
T = 45
np.random.seed(42)
r = 0
bias = 'up'
mag = 0.1
des = f"infinite_real_ratio{r}_bias{bias}_mag{mag}"
At, Qt, At_sd, Qt_sd = simulation(Q,N,n,T,alphas,bias,mag,tp,r,params_1=params_1,params_2=params_2,sd=True)
plot_save_single(At, Qt, des, save=True,limit=True)
plot_save_single_err(At, Qt, At_sd, Qt_sd, des,save=True,limit=True)

Group $j$

In [ ]:
# ratio = 0.1
np.random.seed(42)
r = 0.1
bias = 'up'
mag = 0.1
des = f"real_setting_ratio{r}_bias{bias}_mag{mag}"
At, Qt, At_sd, Qt_sd = simulation(Q,N,n,T,alphas,bias,mag,tp,r,params_1=params_1,params_2=params_2,group='b',sd=True)
plot_save_single(At, Qt, des, True)
plot_save_single_err(At, Qt, At_sd, Qt_sd, des, True)

In [ ]:
# ratio 0.05
np.random.seed(42)
r = 0.05
bias = 'down'
mag = 0.1
des = f"real_setting_ratio{r}_bias{bias}_mag{mag}"
At, Qt, At_sd, Qt_sd = simulation(Q,N,n,T,alphas,bias,mag,tp,r,params_1=params_1,params_2=params_2,group='b',sd=True)
plot_save_single(At, Qt, des,True)
plot_save_single_err(At, Qt, At_sd, Qt_sd, des,True)

In [ ]:
# ratio 0
np.random.seed(42)
r = 0
bias = 'down'
mag = 0.1
des = f"real_setting_ratio{r}_bias{bias}_mag{mag}"
# At, Qt, At_sd, Qt_sd = simulation(Q,N,n,T,alphas,bias,mag,tp,r,params_1=params_1,params_2=params_2,group='b',sd=True)
plot_save_single(At, Qt, des, True)
plot_save_single_err(At, Qt, At_sd, Qt_sd, des, True)

In [ ]:
#infinite horizon to verify the limit
np.random.seed(42)
n = 5
T = 45
r = 0
bias = 'down'
mag = 0.1
des = f"infinite_real_ratio{r}_bias{bias}_mag{mag}"
At, Qt, At_sd, Qt_sd = simulation(Q,N,n,T,alphas,bias,mag,tp,r,params_1=params_1,params_2=params_2,group='b',sd=True)
plot_save_single(At, Qt, des,save=True,limit=True)
plot_save_single_err(At, Qt, At_sd, Qt_sd, des,save=True,limit=True)

Refined retraining process for $i$

In [ ]:
# ratio 0.1
np.random.seed(42)
r = 0.1
bias = 'up'
mag = 0.1
des = f"real_sampler_ratio{r}_bias{bias}_mag{mag}"
At, Qt, At_sd, Qt_sd = simulation(Q,N,n,T,alphas,bias,mag,tp,r,params_1=params_1,params_2=params_2,group='a',sd=True,refined = True)
plot_save_single(At, Qt, des, True)
plot_save_single_err(At, Qt, At_sd, Qt_sd, des, True)

In [ ]:
# ratio 0.05
np.random.seed(42)
r = 0.05
bias = 'up'
mag = 0.1
des = f"real_sampler_ratio{r}_bias{bias}_mag{mag}"
At, Qt, At_sd, Qt_sd = simulation(Q,N,n,T,alphas,bias,mag,tp,r,params_1=params_1,params_2=params_2,group='a',sd=True,refined = True)
plot_save_single(At, Qt, des, True)
plot_save_single_err(At, Qt, At_sd, Qt_sd, des, True)

In [ ]:
# ratio 0
np.random.seed(42)
r = 0
bias = 'up'
mag = 0.1
des = f"real_sampler_ratio{r}_bias{bias}_mag{mag}"
At, Qt, At_sd, Qt_sd = simulation(Q,N,n,T,alphas,bias,mag,tp,r,params_1=params_1,params_2=params_2,group='a',sd=True,refined = True)
plot_save_single(At, Qt, des, True)
plot_save_single_err(At, Qt, At_sd, Qt_sd, des, True)

Refined retraining process for $j$

In [ ]:
# ratio 0.1
np.random.seed(42)
r = 0.1
bias = 'down'
mag = 0
des = f"real_sampler_ratio{r}_bias{bias}_mag{mag}"
At, Qt, At_sd, Qt_sd = simulation(Q,N,n,T,alphas,bias,mag,tp,r,params_1=params_1,params_2=params_2,group='b',sd=True,refined = True)
plot_save_single(At, Qt, des, True)
plot_save_single_err(At, Qt, At_sd, Qt_sd, des, True)

In [ ]:
# ratio 0.05
np.random.seed(42)
r = 0.05
bias = 'down'
mag = 0.1
des = f"real_sampler_ratio{r}_bias{bias}_mag{mag}"
At, Qt, At_sd, Qt_sd = simulation(Q,N,n,T,alphas,bias,mag,tp,r,params_1=params_1,params_2=params_2,group='b',sd=True,refined = True)
plot_save_single(At, Qt, des, True)
plot_save_single_err(At, Qt, At_sd, Qt_sd, des, True)

In [ ]:
# ratio 0
np.random.seed(42)
r = 0
bias = 'down'
mag = 0.1
des = f"real_sampler_ratio{r}_bias{bias}_mag{mag}"
At, Qt, At_sd, Qt_sd = simulation(Q,N,n,T,alphas,bias,mag,tp,r,params_1=params_1,params_2=params_2,group='b',sd=True,refined = True)
plot_save_single(At, Qt, des, True)
plot_save_single_err(At, Qt, At_sd, Qt_sd, des, True)

Unfairness and bias

In [ ]:
# ratio = 0.1
# read i
r = 0.1
bias = 'up'
mag = 0.1
des = f"real_sampler_ratio{r}_bias{bias}_mag{mag}"
Ati_mean, Qti_mean, Ati_sd, Qti_sd = read_results(des)
    
# read j
r = 0.1
bias = 'down'
mag = 0.1
des = f"real_setting_ratio{r}_bias{bias}_mag{mag}"
Atj_mean, Qtj_mean, Atj_sd, Qtj_sd = read_results(des)
    

# Plot
des = f"real_setting_ratio{r}_mag{mag}"
plot_save_fairness(Ati_mean, Atj_mean, Qti_mean, Qtj_mean, des, True)
plot_save_fairness_err(Ati_mean, Atj_mean, Qti_mean, Qtj_mean, Ati_sd, Qti_sd, Atj_sd, Qtj_sd, des, True)

In [ ]:
# ratio = 0.05
# read i
r = 0.05
bias = 'up'
mag = 0.1
des = f"real_sampler_ratio{r}_bias{bias}_mag{mag}"
Ati_mean, Qti_mean, Ati_sd, Qti_sd = read_results(des)
    
# read j
r = 0.05
bias = 'down'
mag = 0.1
des = f"real_setting_ratio{r}_bias{bias}_mag{mag}"
Atj_mean, Qtj_mean, Atj_sd, Qtj_sd = read_results(des)
    

# Plot
des = f"real_setting_ratio{r}_mag{mag}"
plot_save_fairness(Ati_mean, Atj_mean, Qti_mean, Qtj_mean, des, True)
plot_save_fairness_err(Ati_mean, Atj_mean, Qti_mean, Qtj_mean, Ati_sd, Qti_sd, Atj_sd, Qtj_sd, des, True)

In [ ]:
# ratio = 0
# read i
r = 0
bias = 'up'
mag = 0.1
des = f"real_sampler_ratio{r}_bias{bias}_mag{mag}"
Ati_mean, Qti_mean, Ati_sd, Qti_sd = read_results(des)
    
# read j
r = 0
bias = 'down'
mag = 0.1
des = f"real_setting_ratio{r}_bias{bias}_mag{mag}"
Atj_mean, Qtj_mean, Atj_sd, Qtj_sd = read_results(des)
    

# Plot
des = f"real_setting_ratio{r}_mag{mag}"
plot_save_fairness(Ati_mean, Atj_mean, Qti_mean, Qtj_mean, des, True)
plot_save_fairness_err(Ati_mean, Atj_mean, Qti_mean, Qtj_mean, Ati_sd, Qti_sd, Atj_sd, Qtj_sd, des, True)